# EDA Clínico con Sweetviz (NHANES)

Este notebook realiza un análisis exploratorio de datos (EDA) sobre el dataset clínico de NHANES, validando la calidad y distribución de los nuevos biomarcadores.

In [ ]:
# Instalación de dependencias si es necesario
# !pip install sweetviz pandas pyarrow matplotlib seaborn

In [ ]:
import pandas as pd
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
pd.set_option('display.max_columns', None)
%matplotlib inline

## 1. Carga y Preparación de Datos

In [ ]:
DATA_PATH = "../data/02_intermediate/process_data.parquet"

try:
    df = pd.read_parquet(DATA_PATH)
    print(f"Datos cargados correctamente. Dimensiones: {df.shape}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en {DATA_PATH}")
    # Fallback para desarrollo si el path cambia
    # df = pd.read_parquet("../data/processed/NHANES_ULTIMATE_CLEAN.parquet")

In [ ]:
# Mapeo de columnas a nombres en inglés (Biomarcadores)
rename_dict = {
    'Presion_Sistolica': 'SystolicBP',
    'Colesterol_Total': 'TotalCholesterol',
    'Trigliceridos': 'Triglycerides',
    'Glucosa': 'Glucose',
    'Acido_Urico': 'UricAcid',
    'Creatinina': 'Creatinine',
    'Cintura': 'WaistCircumference',
    'TARGET': 'HeartDisease',
    'Sexo': 'Sex',
    'Fumador': 'Smoking',
    'Actividad_Fisica': 'PhysicalActivity'
}

df_english = df.rename(columns=rename_dict)

# Selección de columnas de interés
biomarkers = [
    'SystolicBP', 'TotalCholesterol', 'LDL', 'Triglycerides', 
    'HbA1c', 'Glucose', 'UricAcid', 'Creatinine', 
    'WaistCircumference', 'BMI'
]
categorical_vars = ['Sex', 'Smoking', 'PhysicalActivity', 'HeartDisease']

# Verificar si 'Alcohol' existe, si no, notar su ausencia
if 'Alcohol' in df_english.columns:
    categorical_vars.append('Alcohol')
else:
    print("Advertencia: Variable 'Alcohol' no encontrada en el dataset.")

# Filtrar dataset para el análisis enfocado
cols_to_keep = biomarkers + categorical_vars
# Asegurarnos de que todas las columnas existen
existing_cols = [c for c in cols_to_keep if c in df_english.columns]
df_analysis = df_english[existing_cols].copy()

print("Columnas para análisis:", df_analysis.columns.tolist())

## 2. Validación de Outliers y Estadísticas Descriptivas

Revisamos rápidamente rangos imposibles (ej. Glucosa 0, Presión > 300) antes de generar el reporte visual.

In [ ]:
# Estadísticas descriptivas de biomarcadores numéricos
numeric_cols = [c for c in biomarkers if c in df_analysis.columns]
desc = df_analysis[numeric_cols].describe()
print(desc)

# Chequeo rápido de valores nulos
print("\nValores Nulos:")
print(df_analysis.isnull().sum())

## 3. Análisis de Correlación (Pearson)

Buscamos colinealidad entre variables numéricas.

In [ ]:
plt.figure(figsize=(12, 10))
corr_matrix = df_analysis[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Matriz de Correlación de Biomarcadores")
plt.show()

## 4. Reporte Sweetviz (Target: HeartDisease)

Generamos el reporte comparativo para visualizar la separación entre pacientes sanos y con enfermedad cardíaca.

In [ ]:
target_col = 'HeartDisease'

if target_col in df_analysis.columns:
    # Configuración de Sweetviz
    # pairwise_analysis='off' para eficiencia si hay muchas columnas, aunque aquí hemos filtrado.
    report = sv.analyze(
        df_analysis, 
        target_feat=target_col, 
        pairwise_analysis='off'
    )
    
    output_file = "NHANES_Clinical_Analysis.html"
    report.show_html(output_file)
    print(f"Reporte generado exitosamente: {output_file}")
else:
    print(f"Error: La variable objetivo '{target_col}' no se encuentra en el dataframe.")